Inisiasi

In [1]:
import pandas as pd
from scipy.stats import pointbiserialr

data_train=pd.read_csv('data/data_train.csv')
data_input=pd.read_csv('data/test.csv')

data_result={'id':[],'price_range':[]}
correlation={}
data_min= {}
data_max= {}
columns=data_train.columns
non_numeric = ["blue","dual_sim","four_g","three_g","touch_screen","wifi"]

Korelasi

In [2]:
def find_correlation(corr,data_min,data_max,columns):
    for i in columns:
        if(i in non_numeric):
            correlation, p_value = pointbiserialr(data_train[i], data_train['price_range'])
            corr[i]=correlation
        else :
            corr[i]=data_train[i].corr(data_train['price_range'])
    for i in columns:
        data_min[i]=data_train[i].min()
    for i in columns:
        data_max[i]=data_train[i].max()

In [3]:
find_correlation(correlation,data_min,data_max,columns)

Normalisasi

In [5]:
def normalize(data,data_min,data_max,correlation,columns):
    for i in columns:
        if(i!='price_range'):
            data[i]=10*abs(correlation[i])*(data[i]-data_min[i])/(data_max[i]-data_min[i])

In [6]:
normalize(data_train,data_min,data_max,correlation,columns)
normalize(data_input,data_min,data_max,correlation,columns)

Distance between 2 row

In [7]:
def distance(data1, data2, row1, row2):
    res = 0
    for i in columns:
        if i != 'price_range':
            res += (data1.loc[row1, i] - data2.loc[row2, i]) ** 2
    return res ** (1/2)


Classify Point

In [8]:
def classify_point(input_data,train_data,k,point) :
    arr = []
    for i in range(len(train_data)) :
        arr.append([distance(input_data, train_data, point,i),train_data.loc[i,'price_range']])
    arr.sort(key=lambda x: x[0])
    arr = arr[:k]
    count = [0,0,0,0]
    for i in arr :
        count[i[1]]+=1
    return count.index(max(count))

def accuracy(data_input,data_train,k) :
    count = 0
    for i in range(len(data_input)) :
        if(classify_point(data_input,data_train,k,i)==data_input.loc[i,'price_range']) :
            count+=1
    return count/len(data_input)


Iterate knn with input k

In [9]:
def iterate_k(input_data,train_data,result_data,k) :
    for i in range(len(input_data)):
        result_data['id'].append(i)
        result_data['price_range'].append(classify_point(input_data, train_data, k, i))

Write Output 

In [10]:
def write_output(result_data, file_name) :
    result_data.to_csv(f'data/{file_name}.csv',index=False)

Running KNN

In [11]:
iterate_k(data_input,data_train,data_result,3)

In [ ]:
write_output(pd.DataFrame(data_result))